# 3. fáza projektu
## Matej Horniak, Lukáš Mikula
Dátová sada č. 44

In [7]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
import copy

In [8]:
class DataFrameWrapper():
    def __init__(self, data, person):
        self.data = data
        self.person = person
        self.merged_data = None
        self.pipeline = None
        self.numeric1 = None
        self.numeric2 = None
        self.categoric = None
        self.preproccessors = None
        
    def getMergedData(self):
        # Return DataFrame
        return self.merged_data
    
    def setPersonOtherData(self, data, person):
        # Set new data to data and person DataFrame
        # Other variable set to None
        self.data = data
        self.person = person
        self.merged_data = None
        self.numeric1 = None
        self.numeric2 = None
        self.categoric = None
        return self
    
    def prepocessing(self):
        # Prepocessing data
        # Remove medical_info, merge other and person DataFrame
        # Remove duplicate
        # Modify data to unit format
        
        dict_array = []
        string = '{"mean_glucose": 9999999,"std_glucose": 9999999,"kurtosis_glucose":9999999,"skewness_glucose":9999999,"mean_oxygen":9999999,"std_oxygen":9999999,"kurtosis_oxygen":9999999,"skewness_oxygen":9999999}'
        self.data.medical_info.apply(lambda x: dict_array.append(json.loads(x.replace("'","\"").replace(":\"",":").replace("\",",",").replace("\"}","}") if isinstance(x,str) else string)))
        medical_datas = pd.DataFrame(dict_array)
        
        for item in medical_datas:
            self.data[item] = medical_datas[item].astype(float)
        self.data = self.data.drop(columns=['medical_info'])
        
        self.data = self.data.replace({ 9999999 : np.nan },regex=False)
        self.data = self.data.replace({'^[? ]*$' : np.nan},regex=True)
        self.person = self.person.replace({'^[? ]*$' : np.nan},regex=True)
        self.og = copy.deepcopy(self.data) 
        
        numeric = self.data.select_dtypes(include=['float64'])
        categoric = self.data.select_dtypes(include=['object'])
        numeric['name'] = categoric['name']
        
        numeric = numeric.groupby('name').aggregate('mean')
        categoric = categoric.groupby('name').aggregate('first')
        
        self.data = categoric.merge(numeric, on=['name'])
        
        self.merged_data = pd.merge(self.person, self.data, on=['name','address'], how='left')
        
        self.merged_data = self.merged_data.replace(['FALSE','F'],'f',regex=True)
        self.merged_data = self.merged_data.replace(['TRUE','T'],'t',regex=True)
        
        self.merged_data = self.merged_data.replace([' <=50K'],'<=50K',regex=True)
        self.merged_data = self.merged_data.replace([' >50K'],'>50K',regex=True)
    
    
        self.merged_data = self.merged_data.replace([' Other-relative'],'Other_relative',regex=True)
        self.merged_data = self.merged_data.replace([' Not-in-family'],'Not_in_family',regex=True)
        self.merged_data = self.merged_data.replace([' Own-child'],'Own_child',regex=True)
        
        
        for i, row in self.merged_data.iterrows():
                
            date = parser.parse(self.merged_data.iloc[i]['date_of_birth'].split(' ')[0])
            day = date.day
            month = date.month
            year = date.year
            
            if (year > 2019):
                year = year - 100
        
            self.merged_data.at[i,'date_of_birth'] = str(day) + "/" + str(month) + "/" + str(year)
                    
            if ((pd.isna(row['age'])) | (row['age'] > 115) | (row['age'] < 0)):
                self.merged_data.at[i,'age'] = 2019 - year
        
        return self
    
    
    def removeOutliers(self):
        # Remove outliers on floats datas
        
        ## Remove outliers from 'kurtosis_glucose'
        column = 'kurtosis_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 6) | (self.merged_data[column] < -1)
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'mean_glucose'
        column = 'mean_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 150)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'mean_oxygen'
        column = 'mean_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 120)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'kurtosis_oxygen'
        column = 'kurtosis_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 20)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'skewness_glucose'
        column = 'skewness_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 40)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'skewness_oxygen'
        column = 'skewness_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 350)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'std_glucose'
        column = 'std_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 62) 
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'std_oxygen'
        column = 'std_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 80)  
        self.merged_data.loc[mask ,column] = np.NaN
    
        return self
    
    
    def overtypeData(self):
        # Change data type of object to float64
        # From pipeline
        
        self.merged_data['age'] = self.merged_data['age'].astype(np.float64)
        self.merged_data['education-num'] = self.merged_data['education-num'].astype(np.float64)
        self.merged_data['capital-gain'] = self.merged_data['capital-gain'].astype(np.float64)
        self.merged_data['fnlwgt'] = self.merged_data['fnlwgt'].astype(np.float64)
        # self.merged_data['class'] = self.merged_data['class'].astype(np.float64)
        self.merged_data['hours-per-week'] = self.merged_data['hours-per-week'].astype(np.float64)
        self.merged_data['capital-loss'] = self.merged_data['capital-loss'].astype(np.float64)
        self.merged_data['mean_glucose'] = self.merged_data['mean_glucose'].astype(np.float64)
        self.merged_data['std_glucose'] = self.merged_data['std_glucose'].astype(np.float64)
        self.merged_data['kurtosis_glucose'] = self.merged_data['kurtosis_glucose'].astype(np.float64)
        self.merged_data['skewness_glucose'] = self.merged_data['skewness_glucose'].astype(np.float64)
        self.merged_data['mean_oxygen'] = self.merged_data['mean_oxygen'].astype(np.float64)
        self.merged_data['std_oxygen'] = self.merged_data['std_oxygen'].astype(np.float64)
        self.merged_data['kurtosis_oxygen'] = self.merged_data['kurtosis_oxygen'].astype(np.float64)
        self.merged_data['skewness_oxygen'] = self.merged_data['skewness_oxygen'].astype(np.float64)
        self.merged_data['class'] = self.person['class']
        
        return self
    
    def divideData(self):
        # Divide train data into 3 category
        # Remove age and append on the end of DataFrame
        # Remove class
        
        self.person['class'] = self.merged_data['class']
        self.merged_data = self.merged_data.drop(columns=['class'])
        age_column = copy.deepcopy(self.merged_data['age'])
        self.merged_data = self.merged_data.drop(columns=['age'])
        self.merged_data['age'] = age_column

        numeric = self.merged_data.select_dtypes(include=['float64'])
        self.numeric1 = numeric.iloc[:,:5].columns
        # print(numeric1)
        self.numeric2 = numeric.iloc[:,5:].columns
        # print(numeric2)  
        self.categoric = self.merged_data.select_dtypes(include=['object']).columns
        # print(categoric)
        return self
    
    def createPipelineSteps(self):
        # Create pipeline steps, for categoric mostfrequent strategy
        # For numeric KNN, MissForest
        
        numeric_transformer1 = Pipeline(steps=[('imputer',MissForest(missing_values=np.NaN, max_iter=10, # Nahradenie nulovych hodnot pomocou MissForest
                                                                     decreasing=False, copy=False)), 
                                              ('powerTransformer',PowerTransformer(method='yeo-johnson',
                                                                                   copy=False)) # Transformuje dat aby boli viac z Gaussoveho rozdelenia
                                               ])
        numeric_transformer2 = Pipeline(steps=[('imputer',KNNImputer(missing_values=np.NaN, n_neighbors=5, # Nahradenie nulovych hodnot pomocou KNN
                                                                      weights='distance')), 
                                             # ('RobustScaler',StandardScaler())    # 
                                               ])
        self.categoric_transormer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))]) # Nahradenie nulovych hodnot pomocou najcastejsich hodnot
                                              # ('onehot',OneHotEncoder(handle_unknown='ignore'))])
        preproccesors = ColumnTransformer(transformers=[('Cat', self.categoric_transormer, self.categoric),
                                                        ('Num1', numeric_transformer1, self.numeric1),
                                                        ('Num2', numeric_transformer2, self.numeric2)
                                                        ])
        self.preproccessors = preproccesors
        return self
    
    def appendPipelineSteps(self):
        # Create Pipeline with steps
        
        if self.pipeline is None:
            print("create Pipeline")
            self.pipeline = Pipeline(steps=[('preproccesors', self.preproccessors)])
        else:
            print("append Steps")
            self.pipeline.steps.append([('preproccesors', self.preproccessors)])
        return self
    
    def popSteps(self):
        self.pipeline.steps.pop()
        return self
    
    def Fit(self):
        # Use fit in pipeline
        
        self.pipeline.fit(self.merged_data)
        # self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    
    def FitTransform(self):
        # Use fit and transform in pipeline
        
        pipeline_datas = self.pipeline.fit_transform(self.merged_data)
        self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    
    def Transform(self):
        # Use transform in pipeline
        
        pipeline_datas = self.pipeline.transform(self.merged_data)
        self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    

In [9]:
# Load csv files

data = pd.read_csv("../data/other_train.csv",index_col=0)
person = pd.read_csv("../data/personal_train.csv",index_col=0)


val_data = pd.read_csv("../data/other_valid.csv",index_col=0)
val_person = pd.read_csv("../data/personal_valid.csv",index_col=0)

In [10]:
# Active train data processing

wrapper = DataFrameWrapper(data,person)
wrapper.prepocessing().removeOutliers().divideData().createPipelineSteps().appendPipelineSteps()\
                        .Fit().Transform()



/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and will be removed in 0.2

create Pipeline
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2


In [11]:
# Show training data, and save into trained_datas.csv

merged_data = wrapper.overtypeData().getMergedData()
merged_data.info()
merged_data.to_csv('trained_datas.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 28 columns):
name                3933 non-null object
address             3933 non-null object
sex                 3933 non-null object
date_of_birth       3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3933 non-null object
pregnant            3933 non-null object
relationship        3933 non-null object
education           3933 non-null object
income              3933 non-null object
native-country      3933 non-null object
workclass           3933 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3933 non-null float64
hours-per-week      3933 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3933 non-null float64
kurtosis_oxygen     3933 non-null float64
mean_glucose        3933 non-null float64
mean_oxygen         3933

In [12]:
# Use pipeline to transform on valid data

wrapper.setPersonOtherData(val_data,val_person).prepocessing().removeOutliers().divideData()
wrapper.Transform().overtypeData()


/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and will be removed in 0.23. Don't set `warn_on_dtype` to remove this warning.
  DeprecationWarning)
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and wi

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [14]:
# Show valid data. and save into validate_datas.csv

wrapper.getMergedData().to_csv('validate_datas.csv')
wrapper.getMergedData().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 28 columns):
name                1311 non-null object
address             1311 non-null object
sex                 1311 non-null object
date_of_birth       1311 non-null object
race                1311 non-null object
marital-status      1311 non-null object
occupation          1311 non-null object
pregnant            1311 non-null object
relationship        1311 non-null object
education           1311 non-null object
income              1311 non-null object
native-country      1311 non-null object
workclass           1311 non-null object
education-num       1311 non-null float64
capital-gain        1311 non-null float64
fnlwgt              1311 non-null float64
hours-per-week      1311 non-null float64
capital-loss        1311 non-null float64
kurtosis_glucose    1311 non-null float64
kurtosis_oxygen     1311 non-null float64
mean_glucose        1311 non-null float64
mean_oxygen         1311